#Introdução




Este projeto visa analisar, em período histórico, e prever a demanda no uso de painéis solares em residências no Brasil. Serão utilizadas bases de dados da ANEEL (Agência Nacional de Energia Elétrica) disponibilizados no seguinte site:

https://dadosabertos.aneel.gov.br/organization/agencia-nacional-de-energia-eletrica

Através da base "Relação de empreendimentos de Geração Distribuída", é possível compreender o cenário atual no país em relação a utilização de energia solar em residências e prever a utilização nos próximos 5 anos.

##Importação de bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt

#Exploração dos dados coletados

Através da base disponibilizada pela ANEEL, validaremos os dados disponíveis para as análises.

In [ ]:
!pip install gdown

import gdown

url = 'https://drive.google.com/uc?id=1xJ6tMOwwNHWr_0gnt-ybXVAlH7Nya0zS'  # substitua 'your_file_id' pelo ID do seu arquivo
output = 'file_path'  # substitua 'file_path' pelo caminho onde você deseja salvar o arquivo
gdown.download(url, output, quiet=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/ADM01007 – TÓPICOS ESPECIAIS EM MARKETING II - INTRODUÇÃO A DATA SCIENCE/Trabalho em Grupo/empreendimento-geracao-distribuida.csv", encoding='latin1', sep=";")   # lê o arquivo csv

In [ ]:
df.shape    # retorna a quantidade de linhas e colunas

In [ ]:
df

In [ ]:
df = df[df['SigTipoGeracao'] == 'UFV']    # filtra os dados
df = df[df['DscClasseConsumo'] == 'Residencial']    # filtra os dados
df = df.fillna('')    # preenche os valores nulos

In [ ]:
df.columns

In [ ]:
# Para manter apenas as colunas que vamos utilizar
df = df[['DatGeracaoConjuntoDados','NumCNPJDistribuidora', 'NomAgente', 'CodClasseConsumo', 'codUFibge', 'SigUF', 'CodMunicipioIbge','NomMunicipio', 'NumCPFCNPJ',
         'NomeTitularEmpreendimento', 'CodEmpreendimento', 'DthAtualizaCadastralEmpreend', 'SigModalidadeEmpreendimento',
       'DscModalidadeHabilitado', 'SigTipoGeracao','DscFonteGeracao', 'DscPorte', 'MdaPotenciaInstaladaKW']]

In [ ]:
# Certifique-se de que 'MdaPotenciaInstaladaKW' é numérico
df['MdaPotenciaInstaladaKW'] = df['MdaPotenciaInstaladaKW'].astype(str).str.replace(',', '.').astype(float)

# Converte a coluna 'DthAtualizaCadastralEmpreend' para datetime se ainda não for
df['DthAtualizaCadastralEmpreend'] = pd.to_datetime(df['DthAtualizaCadastralEmpreend'])

# Extrai o ano da coluna 'DthAtualizaCadastralEmpreend'
df['year'] = df['DthAtualizaCadastralEmpreend'].dt.year

# Agrupa por 'year' e calcula a soma de 'MdaPotenciaInstaladaKW'
grouped = df.groupby('year')['MdaPotenciaInstaladaKW'].sum()

print(grouped)

In [ ]:
import statsmodels.api as sm

# Criar um novo DataFrame com a soma de 'MdaPotenciaInstaladaKW' para cada ano
df_grouped = df.groupby('year')['MdaPotenciaInstaladaKW'].sum().reset_index()

# Calcular o somatório acumulado da 'MdaPotenciaInstaladaKW' até cada ano
df_grouped['MdaPotenciaInstaladaKW_cumsum'] = df_grouped['MdaPotenciaInstaladaKW'].cumsum()

# Preparar os dados
X = df_grouped[['year', 'MdaPotenciaInstaladaKW_cumsum']]  # Características (ano e somatório acumulado)
y = df_grouped['MdaPotenciaInstaladaKW']  # Variável alvo (soma de MdaPotenciaInstaladaKW)

# Adicionar uma constante a X
X = sm.add_constant(X)

# Criar um modelo OLS
model = sm.OLS(y, X)

# Ajustar o modelo
results = model.fit()

# Imprimir o resumo das estatísticas do modelo
print(results.summary())

# Definir os anos futuros para os quais você deseja fazer previsões
future_years = [2024, 2025, 2026, 2027]

# Calcular o somatório acumulado para os anos futuros
future_cumsum = [df_grouped['MdaPotenciaInstaladaKW_cumsum'].iloc[-1] + sum(df_grouped['MdaPotenciaInstaladaKW'].iloc[-(i+1):]) for i in range(len(future_years))]

# Adicionar uma constante aos anos futuros e aos somatórios acumulados
future_X = sm.add_constant(pd.DataFrame({'year': future_years, 'MdaPotenciaInstaladaKW_cumsum': future_cumsum}))

# Fazer previsões para os anos futuros
future_preds = results.predict(future_X)

# Imprimir as previsões
for row, pred in zip(future_X.itertuples(), future_preds):
    print(f"Previsão para o ano {row.year} com {round(row.MdaPotenciaInstaladaKW_cumsum, 2)} de potência instalada acumulada: {round(pred, 2)}")

A análise de regressão produz uma série de estatísticas que são usadas para interpretar os resultados. Aqui estão algumas das principais estatísticas e o que elas significam:

1. **R-squared**: Este é o coeficiente de determinação. Ele varia de 0 a 1 e representa a proporção da variância na variável dependente que pode ser prevista a partir da(s) variável(is) independente(s). Neste caso, o R-squared é 0.483, o que significa que cerca de 48.3% da variância em 'MdaPotenciaInstaladaKW' pode ser explicada pelo modelo. Isso é moderadamente bom, mas não excelente.

2. **F-statistic e Prob (F-statistic)**: O F-statistic é uma medida de quão significativo o ajuste do modelo é. Se o Prob (F-statistic) é menor que 0.05, então o modelo é considerado significativo. Neste caso, o Prob (F-statistic) é 0.0138, o que é menor que 0.05, então o modelo é significativo.

3. **coef e P>|t|**: Para cada variável independente, o coeficiente (coef) representa a mudança na variável dependente para uma mudança de uma unidade na variável independente, mantendo todas as outras variáveis constantes. O P>|t| é o p-valor para o teste de hipótese de que o coeficiente é igual a zero (não tem efeito). Se P>|t| é menor que 0.05, então a variável é considerada significativa. Neste caso, tanto 'year' quanto 'MdaPotenciaInstaladaKW_cumsum' têm P>|t| maiores que 0.05, então não são considerados significativos.

4. **Durbin-Watson**: Esta é uma estatística de teste que é usada para detectar a presença de autocorrelação (uma relação entre valores separados de uma mesma variável por um intervalo de tempo específico) nos resíduos de uma análise de regressão. Um valor de 2.0 significa que não há autocorrelação detectada. Neste caso, o valor é 1.194, o que pode indicar a presença de autocorrelação.

5. **Cond. No.**: Este é o número de condição, que é uma medida de quão sensivelmente a função de saída de um modelo muda em relação a uma pequena mudança na entrada. Um número alto pode indicar problemas de multicolinearidade (quando duas ou mais variáveis independentes estão altamente correlacionadas).

Em resumo, embora o modelo como um todo seja significativo, as variáveis individuais não são. Além disso, o R-squared poderia ser maior, e o Durbin-Watson sugere que pode haver autocorrelação nos resíduos. Portanto, você pode querer considerar a possibilidade de modificar o modelo ou usar diferentes variáveis.

In [ ]:
df['count'] = 1

In [ ]:
import statsmodels.api as sm

# Criar um novo DataFrame com a soma de 'count' para cada ano
df_grouped = df.groupby('year')['count'].sum().reset_index()

# Calcular o somatório acumulado da 'count' até cada ano
df_grouped['count_cumsum'] = df_grouped['count'].cumsum()

# Preparar os dados
X = df_grouped[['year', 'count_cumsum']]  # Características (ano e somatório acumulado)
y = df_grouped['count']  # Variável alvo (soma de count)

# Adicionar uma constante a X
X = sm.add_constant(X)

# Criar um modelo OLS
model = sm.OLS(y, X)

# Ajustar o modelo
results = model.fit()

# Imprimir o resumo das estatísticas do modelo
print(results.summary())

# Definir os anos futuros para os quais você deseja fazer previsões
future_years = [2024, 2025, 2026, 2027]

# Calcular o somatório acumulado para os anos futuros
future_cumsum = [df_grouped['count_cumsum'].iloc[-1] + sum(df_grouped['count'].iloc[-(i+1):]) for i in range(len(future_years))]

# Adicionar uma constante aos anos futuros e aos somatórios acumulados
future_X = sm.add_constant(pd.DataFrame({'year': future_years, 'count_cumsum': future_cumsum}))

# Fazer previsões para os anos futuros
future_preds = results.predict(future_X)

# Imprimir as previsões
for row, pred in zip(future_X.itertuples(), future_preds):
    print(f"Previsão para o ano {row.year} com {round(row.count_cumsum, 2)} de contagem acumulada: {round(pred, 2)}")

A análise de regressão produz uma série de estatísticas que são usadas para interpretar os resultados. Aqui estão algumas das principais estatísticas e o que elas significam:

1. **R-squared**: Este é o coeficiente de determinação. Ele varia de 0 a 1 e representa a proporção da variância na variável dependente que pode ser prevista a partir da(s) variável(is) independente(s). Neste caso, o R-squared é 0.486, o que significa que cerca de 48.6% da variância em 'count' pode ser explicada pelo modelo. Isso é moderadamente bom, mas não excelente.

2. **F-statistic e Prob (F-statistic)**: O F-statistic é uma medida de quão significativo o ajuste do modelo é. Se o Prob (F-statistic) é menor que 0.05, então o modelo é considerado significativo. Neste caso, o Prob (F-statistic) é 0.0132, o que é menor que 0.05, então o modelo é significativo.

3. **coef e P>|t|**: Para cada variável independente, o coeficiente (coef) representa a mudança na variável dependente para uma mudança de uma unidade na variável independente, mantendo todas as outras variáveis constantes. O P>|t| é o p-valor para o teste de hipótese de que o coeficiente é igual a zero (não tem efeito). Se P>|t| é menor que 0.05, então a variável é considerada significativa. Neste caso, tanto 'year' quanto 'count_cumsum' têm P>|t| maiores que 0.05, então não são considerados significativos.

4. **Durbin-Watson**: Esta é uma estatística de teste que é usada para detectar a presença de autocorrelação (uma relação entre valores separados de uma mesma variável por um intervalo de tempo específico) nos resíduos de uma análise de regressão. Um valor de 2.0 significa que não há autocorrelação detectada. Neste caso, o valor é 1.126, o que pode indicar a presença de autocorrelação.

5. **Cond. No.**: Este é o número de condição, que é uma medida de quão sensivelmente a função de saída de um modelo muda em relação a uma pequena mudança na entrada. Um número alto pode indicar problemas de multicolinearidade (quando duas ou mais variáveis independentes estão altamente correlacionadas).

Em resumo, embora o modelo como um todo seja significativo, as variáveis individuais não são. Além disso, o R-squared poderia ser maior, e o Durbin-Watson sugere que pode haver autocorrelação nos resíduos. Portanto, você pode querer considerar a possibilidade de modificar o modelo ou usar diferentes variáveis.


In [ ]:
# Converte a coluna 'DthAtualizaCadastralEmpreend' para datetime se ainda não for
df['DthAtualizaCadastralEmpreend'] = pd.to_datetime(df['DthAtualizaCadastralEmpreend'])

# Extrai o ano da coluna 'DthAtualizaCadastralEmpreend'
df['year'] = df['DthAtualizaCadastralEmpreend'].dt.year

# Conta o número de atualizações em cada ano
updates_per_year = df.groupby('year').size()

# Mostra o resultado
print(updates_per_year)

In [ ]:
import statsmodels.api as sm

# Filtrar o DataFrame para considerar apenas os anos acima de 2019
df_filtered = df[df['year'] > 2019]

# Criar um novo DataFrame com a soma de 'count' para cada ano
df_grouped = df_filtered.groupby('year')['count'].sum().reset_index()

# Calcular o somatório acumulado da 'count' até cada ano
df_grouped['count_cumsum'] = df_grouped['count'].cumsum()

# Preparar os dados
X = df_grouped[['year', 'count_cumsum']]  # Características (ano e somatório acumulado)
y = df_grouped['count']  # Variável alvo (soma de count)

# Adicionar uma constante a X
X = sm.add_constant(X)

# Criar um modelo OLS
model = sm.OLS(y, X)

# Ajustar o modelo
results = model.fit()

# Imprimir o resumo das estatísticas do modelo
print(results.summary())

# Definir os anos futuros para os quais você deseja fazer previsões
future_years = [2024, 2025, 2026, 2027]

# Calcular o somatório acumulado para os anos futuros
future_cumsum = [df_grouped['count_cumsum'].iloc[-1] + sum(df_grouped['count'].iloc[-(i+1):]) for i in range(len(future_years))]

# Adicionar uma constante aos anos futuros e aos somatórios acumulados
future_X = sm.add_constant(pd.DataFrame({'year': future_years, 'count_cumsum': future_cumsum}))

# Fazer previsões para os anos futuros
future_preds = results.predict(future_X)

# Imprimir as previsões
for row, pred in zip(future_X.itertuples(), future_preds):
    print(f"Previsão para o ano {row.year} com {round(row.count_cumsum, 2)} de contagem acumulada: {round(pred, 2)}")

In [ ]:
import matplotlib.pyplot as plt

# Criar um novo DataFrame com as previsões
df_preds = pd.DataFrame({'year': future_years, 'MdaPotenciaInstaladaKW_cumsum': future_cumsum, 'prediction': future_preds})

# Concatenar os dados originais com as previsões
df_total = pd.concat([df_grouped, df_preds], ignore_index=True)

# Plotar os dados originais
plt.plot(df_total['year'], df_total['MdaPotenciaInstaladaKW_cumsum'], label='Dados originais')

# Plotar as previsões
plt.plot(df_total['year'], df_total['prediction'], label='Previsões')

# Adicionar um título e rótulos aos eixos
plt.title('Previsões de MdaPotenciaInstaladaKW')
plt.xlabel('Ano')
plt.ylabel('MdaPotenciaInstaladaKW_cumsum')

# Adicionar uma legenda
plt.legend()

# Mostrar o gráfico
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Criar um novo DataFrame com as previsões
df_preds = pd.DataFrame({'year': future_years, 'MdaPotenciaInstaladaKW_cumsum': future_cumsum, 'prediction': future_preds})

# Concatenar os dados originais com as previsões
df_total = pd.concat([df_grouped, df_preds], ignore_index=True)

# Plotar os dados originais
plt.plot(df_total['year'], df_total['MdaPotenciaInstaladaKW_cumsum'], label='Dados originais', marker='o', linestyle='-')

# Plotar as previsões
plt.plot(df_total['year'], df_total['prediction'], label='Previsões', marker='o', linestyle='--')

# Adicionar um título e rótulos aos eixos
plt.title('Previsões de MdaPotenciaInstaladaKW Acumulada')
plt.xlabel('Ano')
plt.ylabel('MdaPotenciaInstaladaKW Acumulada')

# Definir os rótulos do eixo x para todos os anos
plt.xticks(df_total['year'], rotation=45)

# Adicionar uma legenda
plt.legend()

# Mostrar o gráfico
plt.show()